## Deliverable 2. Create a Customer Travel Destinations Map.

In [77]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)


In [78]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Country,Date
0,Upernavik,72.7868,-56.1549,10.98,86,58,7.87,broken clouds,GL,2021-01-17 22:24:27
1,Rikitea,-23.1203,-134.9692,78.91,75,19,14.74,few clouds,PF,2021-01-17 22:24:27
2,Bluff,-46.6000,168.3333,60.01,56,69,8.01,broken clouds,NZ,2021-01-17 22:24:27
3,Katsuura,35.1333,140.3000,39.00,69,95,4.00,overcast clouds,JP,2021-01-17 22:24:27
4,Leshukonskoye,64.8989,45.7622,-7.11,91,92,3.44,overcast clouds,RU,2021-01-17 22:24:27


In [79]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature that you would like for your trip?"))
max_temp = float(input("What is the maximum temperature that you would like for your trip?"))

What is the minimum temperature that you would like for your trip?75
What is the maximum temperature that you would like for your trip?90


In [80]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_hotel_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"]>=min_temp)]

In [81]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_hotel_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""

In [82]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
        lat = row["Lat"]
        lng = row["Lng"]
        
        params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
        hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
        except:
            print(f"Hotel not found, skipping city.")
        
            
        

Hotel not found, skipping city.
Hotel not found, skipping city.
Hotel not found, skipping city.
Hotel not found, skipping city.
Hotel not found, skipping city.
Hotel not found, skipping city.
Hotel not found, skipping city.
Hotel not found, skipping city.


In [83]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'] = hotel_df['Hotel Name'].replace('', np.nan)

hotel_df = hotel_df.dropna()

In [84]:
# 8a. Create the output File (CSV)
hotel_df.to_csv('Vacation_Search\WeatherPy_vacation.csv')
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv('Vacation_Search\City_Data', index_label="City_ID")

In [85]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [86]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))